### Ce travail a été réalisé à l'aide de :
    1- https://pypi.org/project/selenium/
    2- CHATGPT
    3- https://www.selenium.dev/documentation/
    4- Google
 

# Etape 1 : Installer selenium
     1- Ouvrir prompt Anaconda 
     2- copier/coller ce pip (/!\ attention il faut la dernière version) ==> pip install -U selenium
     3- Entrer

# Etape 2 : Importer les Libraries requises 

In [1]:
# le module webdriver : permet de naviger sur internet (intéragire avec une page via le navigateur)
from selenium import webdriver

# le module By : permet a selenium de trouver un contenu spécifique dans un page web 
from selenium.webdriver.common.by import By

# la librarie WebDriverWait : permet à selenium d'attendre jusqu'à ce que les éléments de la page web s'affiche correctement.
from selenium.webdriver.support.ui import WebDriverWait

# le module expected_conditions : Ce module contient une collection de conditions que WebDriver peut attendre jusqu'à ce qu'elles se 
# réalisent avant de poursuivre l'exécution du code. On va l'utiliser avec WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# le module time : est utilisée pour effectuer des opérations liées au temps. Dans notre cas on souhaite mettre en pause 
# l'exécution de notre programme pendant un certain nombre de secondes.
import time

# le module BeautifulSoup: est utilisée pour extraire et manipuler les données à partir de fichiers HTML et XML. 
from bs4 import BeautifulSoup

# le module pandas: est utilisée pour la manipulation et l'analyse de données (exemple créer un DataFrame...)
import pandas as pd


# Etape 3 : Se connecter au compte LinkedIn via Selenium:

In [2]:
# Ouvrir le navigateur Firefox: le navigateur va s'ouvrir automatiquement.
browser = webdriver.Firefox()


#  Accédez à la page login de LinkedIn : get, permet de chercher le lien souhaité
url_loging= browser.get('https://www.linkedin.com/login')


# Saisir automatiquement l'adresse mail : 'A ne pas oublier de saisir l'adresse mail et le mot de passe ci-dessous '
    # 1- find_element(By.ID..) : il va chercher sur la page web l'élément par l'ID "username":
        #"""faire une capture d'acran du code source a quoi resemble cet élément"""
    # 2- .send_keys : ensuite cette méthode permet de saisir une chaine de caractére dans cet élément. 
                     # Donc il va saisir notre adresse mail dans le champs de saisie.
browser.find_element(By.ID,'username').send_keys('XXX@XXX.com')


# Saisir automatiquement le mot de passe : même logique que l'adresse mail.
browser.find_element(By.ID,'password').send_keys('XXXXXX')


# Cliquer automatiquement sur le boutton Connexion pour se connecter :
    # CSS_SELECTOR : permet de spécifier quel élément trouver. 
    # Dans ce cas,on va chercher le bouton qui se trouve à l'intérieur d'un élément ayant la classe login__form_action_container
            #"""faire une capture d'acran du code source a quoi resemble cet élément"""
    # la méthode click(): est appelée pour simuler un clic de souris sur cet élément.
browser.find_element(By.CSS_SELECTOR,'.login__form_action_container button').click()

# Etape 4 : Extraction des données via Bs4/Selenium:
    # attention, il ne faut pas fermer le navigateur Firefox qui vient de s'ouvrir automatiquement (il faut rester connecté)

In [5]:
# Nombre total d'emploi: l'objectif est de calculé le nombre de page à itérer dans la boucle ci-dessous
browser.get("https://www.linkedin.com/jobs/search/?keywords=Data%20Analyst&location=France&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&sortBy=DD")
s = BeautifulSoup(browser.page_source,'html')
nbr_jobs = int(s.find('div', class_='jobs-search-results-list__subtitle').get_text(strip=True).replace('\u202f','').replace(' résultats',''))
print(nbr_jobs)



# Fonction pour scroller la page de manière plus lente (comme un être humain).
# L'objectif est de s'assurer que la page est complétement chargée. 
# Car sans cette fonction, le code récupére que les 7 premiers emplois. La cause, c'est que la page n'a pas été chargé à 100%

# def slow_scroll(element, distance): Cette ligne déclare une nouvelle fonction appelée slow_scroll prenant deux paramètres : 
    # element: qui est l'élément à faire défiler (dans notre cas on souhate défiler la liste des emplois et non pas la déscription de l'emploi)
    # distance: qui est la distance de défilement souhaitée.
# """faire une capture d'ecran de la page des emplois pour montrer les deux éléments (liste des job et la déscription)"""
def slow_scroll(element, distance):
    
    # Cette valeur de 10 représente le nombre de pixels par lesquels l'élément sera déplacé à chaque étape du défilement.
    # pour info, La distance parcourue lors du défilement d'une page web est mesurée en pixels.
    # Plus la variable scroll_step est grand, plus le défilement sera rapide, et vice versa.
    scroll_step = 10

    # création d'une Boucle pour faire défiler l'élément par petits pas: c'est à dire itère de 0 jusqu'à la distance, avec un pas de scroll_step pixels
    # cette distance souhaité sera spécifier en bas de ce code.
    # pour info, le tiret du bas/underscore ( _ ) est utilisé comme une variable pour indiquer que cette variable de boucle n'est pas utilisée dans le corps de la boucle et n'est donc pas importante. 
    for _ in range(0, distance, scroll_step):
        
        # Faire défiler l'élément par scroll_step pixels : À chaque itération de la boucle, cette ligne exécute du code JavaScript via Selenium pour faire défiler l'élément de 10 pixels (voir la variable scroll_step)
        # vers le bas. 
            # arguments[0] : fait référence à "element" spécifié (qui est dans notre cas c'est la variable "jobs_list_element" défini en bas de ce code). 
            # arguments[1] : fait référence à scroll_step.
        # Cela permet d'executer le défilement à l'élément spécifié ( qui est la variable "jobs_list_element" défini en bas de ce code)
        browser.execute_script("arguments[0].scrollBy(0, arguments[1]);", element, scroll_step)

        # Pause courte entre chaque mouvement pour simuler un défilement plus lent : 
        time.sleep(0.01)

# Liste vide pour stocker les données.
job_info =[]

# Cette boucle permet de passer à la page suivante afin d'extraire les données de chaque page.
# pour info:
    # chaque page contient 25 emplois.
    # les numéro des page sont organisés de cette manière:
        # page 1 = de 0-24 ==> donc start page = 0
        # page 2 = de 25-49 ==> donc start page = 25
        # page 3 = de 50-74 ==> donc start page = 50
        # ...
        # la page 40 (j'ai l'impression que c'est la page max, je ne trouve pas 41...) ==> donc la dernière start page = 975
    # dans notre cas pour extraire toutes le page on va mettre 1000 (afin que la boucle s'arrête à 975)
for i in range(0,25,25):
    
    # Accédez à la page d'emploi LinkedIn : les filtres sont déjà dans le lien :
        # data analyst, France, les données du mois dérnier, trier par le plus récents. 
    # Pour choisir les filtres ==> Il suffit de se connecter, choisir les filtres qu'on souhaite ensuite copier le lien.
    browser.get("https://www.linkedin.com/jobs/search/?keywords=Data%20Analyst&location=France&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&sortBy=DD&start="+str(i))

    # Trouver l'élément contenant la liste des emplois: 
        # 1- find_element(By.CLASS_NAME..) : il va chercher sur la page web l'élément par l'CLASS_NAME 'jobs-search-results-list':
        #"""faire une capture d'acran du code source a quoi resemble cet élément + la page d'emploi linkedin"""
    jobs_list_element = browser.find_element(By.CLASS_NAME,'jobs-search-results-list')

    # Faires défiler l'élément de la liste des emplois jusqu'à ce que toutes les offres d'emploi soient chargées.
    # donc la condition 'True' permet à la boucle de continuer à faire défiler la liste des emplois jusqu'à ce que toutes les offres d'emploi soient chargées,
    # puis elle s'arrêtera lorsque le bouton "Voir plus" ne peut plus être trouvé ou cliqué.
    while True:
        
        # Scroll jusqu'en bas de l'élément de la liste des emplois:
            # Cette ligne appelle la fonction 'slow_scroll' (définit ci-dessus) pour faire défiler lentement l'élément de la liste des emplois 
                # ==> voir la variable 'jobs_list_element' définit ci-dessus. 
            # 4000 pixels est la distance totale qu'on souhaite faire défiler dans chaque page.
        slow_scroll(jobs_list_element,4000)

        # Attendre un court délai pour permettre aux offres d'emploi supplémentaires de se charger.
        time.sleep(2)

        # Vérifier s'il y a un bouton "Voir plus" pour charger plus d'offres d'emploi:
            # Try : fonction comme la condition 'if'. Si le bouton "voir plus" existe dans ce cas 'click' sur ce bouton
        try:
            
            # Cette ligne utilise WebDriverWait, une fonctionnalité de Selenium qui attend un certain 
            # temps jusqu'à ce qu'une condition spécifique soit remplie.
                # WebDriverWait (browser, 5) : Cela signifie que le WebDriverWait attendra jusqu'à 5 secondes au maximum pour que la condition spécifiée soit remplie.
                # until() : contient la codition dont WebDriverWait() doit attendre.
                # EC.element_to_be_clickable(...) : C'est la condition que WebDriverWait doit attendre. c'est à dire si le bouton cliquable 'voir plus' est identifié le code va cliquer.
            # En résumé, ce code attend jusqu'à 5 secondes que le bouton "Voir plus" devienne cliquable dans le navigateur. Une fois que le bouton est cliquable ou 
            # que le délai d'attente est écoulé, le code continue son exécution.
            more_button = WebDriverWait(browser, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'Voir plus')]")))
            more_button.click()
        
            # sinon, sortir de la boucle si le bouton "Voir plus" n'est pas trouvé ou s'il n'est pas cliquable.
        except:
            break  

    # Récuperer le code source HTML:
        # BeautifulSoup:est une bibiothéque pour analyser le code source HTML d'une page web. elle prend 2 arguments:
            # browser.page_source : Cette partie récupère le code source HTML de la page web actuellement ouverte dans le navigateur (que j'ai nommé browser)
            # 'html': Cet argument indique à BeautifulSoup le type de parser à utiliser pour analyser le code HTML.
    s = BeautifulSoup(browser.page_source,'html')

    # stocker dans une variable toutes :
        # les balises 'div' 
        # qui ont la class 'flex-grow-1 artdeco-entity-lockup__content ember-view'
    # Ces Classes (se sont les blocs d'emploi) contient toutes les info sur l'emploi :
        # """faire une capture d'cran du bloc d'emploi"""
    # L'objectif est de DECOUPER LE CODE HTML afin de facilité la recherche des info c'est à dire on va récupérer la partie code HTML qui nous intéresse au lieu de rechercher dans tout le code source.
    job = s.find_all('div', class_ = 'flex-grow-1 artdeco-entity-lockup__content ember-view')

    # Stocker dans une variable toutes :
        # les balises 'ul' 
        # qui ont la class 'jon-card-list....'
    # Ces Classes contient les dates de publication des emplois :
        # """faire une capture d'cran du bloc d'emploi"""
    # L'objectif est de DECOUPER LE CODE HTML afin de facilité la recherche des info c'est à dire on va récupérer la partie code HTML qui nous intéresse au lieu de rechercher dans tout le code source.
    date = s.find_all('ul', class_=('job-card-list__footer-wrapper job-card-container__footer-wrapper flex-shrink-zero display-flex t-sans t-12 t-black--light t-normal t-roman'))

    # iteration dans les 2 variables ci-dessous pour récuperer les données dont on aura besoin:
    for j,t in zip(job,date):

        # Extraction du titre de l'emploi: 
            # j.find : Ici, nous allons chercher touts les éléments <strong> qui se trouve dans la variable "job" déclaré ci-dessus. 
            # .text : Ensuite nous allons récupérer que le texte de cette balise
            # .strip() : Permets de supprimer les espaces qui se trouve au bédut et à la fin d'un text
                # """faire une capture d'ecran du code HTML"""
        job_title = j.find('strong').text.strip()

        # Extraction du nom de la société :
            # même logique que le job_title. Sauf qu'ici on cherche toutes les class 'job-card...'
                # """faire une capture d'ecran du code HTML"""
        compagny = j.find(class_='job-card-container__primary-description').text.strip()

        # Extraction de la localisation de l'emploi:
            # même logique que le job_title. Sauf qu'ici on cherche toutes les balises 'li'
                # """faire une capture d'ecran du code HTML"""
        location = j.find('li').text.strip()
        
        # Extraction du lien de l'emploi:
        link = j.a['href']
        

        # Extraction de la date de publication de l'emploi avec une Condition if (Si la date est NONE donc le code va retourner "Aucune date Trouvée.)
            # même logique que le job_title. Sauf qu'ici on cherche touts les éléments <time> de la variable "date" déclaré ci-dessus.
                # """faire une capture d'ecran du code HTML + le bloc d'emploi"""
        if t.find('time') is not None:
           
            # Extraction de la date de la pub : si la date existe dans ce cas, le code extrait la valeur de l'attribut 'datetime' de l'élément <time>
                # """faire une capture d'ecran de bloc d'emploi"""
            date = t.time['datetime']
            
            # Extraction de la durée entre la date de la pub et la date d'aujourd'hui : Ensuite le code extrait également le texte de cet élément
                # """faire une capture d'ecran du bloc d'emploi"""
            duree = t.time.get_text(strip=True)[0:12]
        
        # sinon le code va retourner "Aucune date Trouvée." si l'élément <time> = None
        else:
            date ="Aucune date Trouvée."
            duree ="Aucune date Trouvée."      

        # Création d'un dictionnaire pour stocker les données extraites ci-dessus. L'objectif est de les stocker dans la liste "job_info" déclarer au début du code
        job  = {'Job_Title' : job_title, 'Company': compagny,'Location': location, 'Job_posted':date, 'Duration':duree, 'Link':link}

        # Stocker le dictionnaire ci-dessus dans la liste "job_info". L'objectif de cette liste, est de créer un DF.
        job_info.append(job)

# Création du DataFrame
df = pd.DataFrame(job_info)

# Séparer la colonne 'Location' pour créer les colonnes Ville, region et pays (délimiteur = ',')
# exemple ==> Malakoff, Île-de-France, France (Hybride)
df[['City', 'Region', 'Pays']] = df['Location'].str.split(', ', expand=True)

# Remplacer les valeurs vide de la colonne 'Pays' et 'Region' par la valeur de la colonne 'Ville'
df['Pays'].fillna(df['City'], inplace=True)
df ['Region'].fillna(df['City'], inplace=True)

# Séparer la colonne 'Pays' pour créer 2 colonnes Pyas et Work_mode (À distanc, sur site..) ==> délimiteur = "(" . 
# Exemple ==> France (Hybride) 
df[['Pays', 'Work_Mode']] = df['Pays'].str.split('(', expand=True)

# Ensuite supprimer la parentaise qui reste.
# Exemple ==> Hybride)
df['Work_Mode'] = df['Work_Mode'].str.replace(')',"",regex=False)

 # Supprimer la colonne 'Localisation' car on va plus l'utiliser       
df.drop(columns=['Location'], inplace=False)

# Ajouter 'www.linkedin.com' au début du lien. Exemple "/jobs/view/3922229798/?"
df['Link'] ='https://www.linkedin.com'+ df['Link']

# Réorganiser les colonnes du Df:
df = df.reindex(columns=['Job_Title','Company','City','Region','Pays','Work_Mode','Job_posted','Duration', 'Link'])

# shape : permets de connaître la taille du Df: 
# exemple ==> (25,9) il y a 25 lignes et 9 colonnes
print(df.shape)

# Affichier les 7 premières lignes :
df.head(7)

1462
(25, 9)


,Job_Title,Company,City,Region,Pays,Work_Mode,Job_posted,Duration,Link
0,Biostatistician H/F,Ividata Life Sciences,Paris,Île-de-France,France,Hybride,Aucune date Trouvée.,Aucune date Trouvée.,https://www.linkedin.com/jobs/view/3929631620/...
1,Consultant Senior en Quant - Bdo H/F,BDO France,Paris,Île-de-France,France,Hybride,2024-05-18,Il y a 3 heu,https://www.linkedin.com/jobs/view/3929635193/...
2,Consultant Senior en Quant - Bdo H/F,BDO France,Paris,Île-de-France,France,Hybride,2024-05-18,Il y a 3 heu,https://www.linkedin.com/jobs/view/3929634357/...
3,Consultant Senior en Quant - Bdo H/F,BDO France,Paris,Île-de-France,France,Hybride,2024-05-18,Il y a 3 heu,https://www.linkedin.com/jobs/view/3929637057/...
4,Business Intelligence Analyst - (H/F) - En Alt...,Jooble,Saint-Denis,Île-de-France,France,Sur site,2024-05-18,Il y a 3 heu,https://www.linkedin.com/jobs/view/3929632365/...
5,Business Intelligence Analyst - (H/F) - En Alt...,Jooble,Lyon,Auvergne-Rhône-Alpes,France,Sur site,2024-05-18,Il y a 3 heu,https://www.linkedin.com/jobs/view/3929629935/...
6,Business Intelligence Analyst - (H/F) - En Alt...,Jooble,Nantes,Pays de la Loire,France,Sur site,2024-05-18,Il y a 3 heu,https://www.linkedin.com/jobs/view/3929629797/...


### Pour aller plus loin dans l'analyse : Ajouter la description de l'emploi pour faire du textmining. Par exemple:
      ==> Extraire les compétences demandées par l'employeur
      ==> Extraire les avantages
      ==> Salaire...

In [44]:
## solution: ouvrir chaque lien du df et extraire la déscription de chaque page:
# l'inconvenient de cette méthode c'est qu'elle ouvre lien par lien. 
# voi la solution "Extraction_Job_Title_LinkedIn_With_Account 2" qui ouvre emploi par emploi dans la même page (A finir) 
Descption = []
for n in df['Link']:
    browser.get(n)
    time.sleep(3)
    WebDriverWait(browser,5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label,'Cliquez ici pour voir plus de description')]"))).click()
    s = BeautifulSoup(browser.page_source, 'html')
    descp = s.find_all('div', class_='jobs-description__content jobs-description-content jobs-description__content--condensed')
    for n in descp:
        job_desp = n.find(class_='mt4').get_text(separator='\n', strip=True)
        Descption.append(job_desp)
    
df['Detail'] = Descption
df

,Job_Title,Company,City,Region,Pays,Work_Mode,Job_posted,Duration,Link,Detail
0,Biostatistician H/F,Ividata Life Sciences,Paris,Île-de-France,France,Hybride,Aucune date Trouvée.,Aucune date Trouvée.,https://www.linkedin.com/jobs/view/3929631620/...,Ividata Life Sciences est la filiale du groupe...
1,Consultant Senior en Quant - Bdo H/F,BDO France,Paris,Île-de-France,France,Hybride,2024-05-18,Il y a 3 heu,https://www.linkedin.com/jobs/view/3929635193/...,Evoluez au côté de talents passionnés !\n5ème ...
2,Consultant Senior en Quant - Bdo H/F,BDO France,Paris,Île-de-France,France,Hybride,2024-05-18,Il y a 3 heu,https://www.linkedin.com/jobs/view/3929634357/...,Evoluez au côté de talents passionnés !\n5ème ...
3,Consultant Senior en Quant - Bdo H/F,BDO France,Paris,Île-de-France,France,Hybride,2024-05-18,Il y a 3 heu,https://www.linkedin.com/jobs/view/3929637057/...,Evoluez au côté de talents passionnés !\n5ème ...
4,Business Intelligence Analyst - (H/F) - En Alt...,Jooble,Saint-Denis,Île-de-France,France,Sur site,2024-05-18,Il y a 3 heu,https://www.linkedin.com/jobs/view/3929632365/...,We are promoting this job opportunity as provi...
5,Business Intelligence Analyst - (H/F) - En Alt...,Jooble,Lyon,Auvergne-Rhône-Alpes,France,Sur site,2024-05-18,Il y a 3 heu,https://www.linkedin.com/jobs/view/3929629935/...,We are promoting this job opportunity as provi...
6,Business Intelligence Analyst - (H/F) - En Alt...,Jooble,Nantes,Pays de la Loire,France,Sur site,2024-05-18,Il y a 3 heu,https://www.linkedin.com/jobs/view/3929629797/...,We are promoting this job opportunity as provi...
7,Business Intelligence Analyst - (H/F) - En Alt...,Jooble,Saint-Léopardin-d'Augy,Auvergne-Rhône-Alpes,France,Sur site,2024-05-18,Il y a 3 heu,https://www.linkedin.com/jobs/view/3929627896/...,We are promoting this job opportunity as provi...
8,Business Intelligence Analyst - (H/F) - En Alt...,Jooble,Boulogne-Billancourt,Île-de-France,France,Sur site,2024-05-18,Il y a 3 heu,https://www.linkedin.com/jobs/view/3929631244/...,We are promoting this job opportunity as provi...
9,Alternance - Data Anlayst H/F,Eneria,Montlhéry,Île-de-France,France,Sur site,2024-05-18,Il y a 3 heu,https://www.linkedin.com/jobs/view/3929624872/...,"CET-FR-ENERIA\nEneria, filiale du Groupe Monno..."
